In [ ]:
import numpy as np # For numerical computation
import pandas as pd # Data manipulation
import re
import os # system-wide functions
import h5py
# import scipy.io # reading matlab files in python
# from scipy import signal #signal processing
# from scipy.fftpack import fft, dct #signal processing

# from sklearn.linear_model import LinearRegression #linear regression model
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import KFold, train_test_split # cross validation split
# from sklearn.metrics import mean_squared_error
# import warnings
# warnings.filterwarnings('ignore')

# from matplotlib import pyplot as plt # For plotting graphs(Visualization)
# from collections import Counter


In [ ]:
for i in range(390,559):
    dict={}
    j = f"{i:03d}"
    file_path = f"E:/CNR_2025/data/external/PulseDB/PulseDB_Vital/p000{j}.mat"
    output_path = f"E://CNR_2025//data//raw//PulseDB//PulseDB_Vital//p000{j}.csv"

    if not os.path.exists(file_path):
        continue
    

    try:
        sample_file =h5py.File(file_path, 'r')['Subj_Wins']
        for col in range(len(list(sample_file.keys()))):
            s = list(sample_file.values())[col]
            l = list(sample_file.keys())[col]

            dereferenced_content = []

            for ref in s[0]:
                # Dereference the object reference
                dereferenced_data = s.file[ref]
                
                # Extract the content from the dereferenced object
                dereferenced_content.append(dereferenced_data[()])
                
            merged_array = np.concatenate(dereferenced_content, axis=1).flatten()
            merged_list = merged_array.tolist()
            dict[s]=merged_list
        max_len = max(len(v) for v in dict.values())

        for k, v in dict.items():
            if len(v) < max_len:
                # Find the most frequent value
                most_common = Counter(v).most_common(1)[0][0]
                # Calculate how many to add
                to_add = [most_common] * (max_len - len(v))
                # Extend the list
                dict[k] = v + to_add

        df = pd.DataFrame(dict)
        df.to_csv(output_path, index=False)
        print(f"Saved: {output_path}")

    except Exception as e:
        print(f"Skipping {file_path} due to error: {e}")




In [ ]:
Saved: E://CNR_2025//data//raw//PulseDB//PulseDB_Vital//p000388.csv

In [ ]:

sample_file =h5py.File('E:/CNR_2025/data/external/PulseDB/PulseDB_Vital/p000381.mat', 'r')['Subj_Wins']
dict={}
for col in range(len(list(sample_file.keys()))):
    s = list(sample_file.values())[col]
    l = list(sample_file.keys())[col]

    dereferenced_content = []

    for ref in s[0]:
        # Dereference the object reference
        dereferenced_data = s.file[ref]
        
        # Extract the content from the dereferenced object
        dereferenced_content.append(dereferenced_data[()])
        
    merged_array = np.concatenate(dereferenced_content, axis=1).flatten()
    merged_list = merged_array.tolist()
    dict[s]=merged_list
max_len = max(len(v) for v in dict.values())

for k, v in dict.items():
    if len(v) < max_len:
        # Find the most frequent value
        most_common = Counter(v).most_common(1)[0][0]
        # Calculate how many to add
        to_add = [most_common] * (max_len - len(v))
        # Extend the list
        dict[k] = v + to_add
        print(f"Padded key '{k}' with {len(to_add)} x '{most_common}'")

df = pd.DataFrame(dict)
output_path = f"E://CNR_2025//data//raw//PulseDB//PulseDB_Vital//p000381.csv"
df.to_csv(output_path, index=False)
print(f"Saved: {output_path}")


In [ ]:
for i in range(500,570):
    dict={}
    j = f"{i:03d}"
    file_path = f"E://CNR_2025//data//raw//PulseDB//PulseDB_Vital//p000{j}.csv"
    output_path = f"E://CNR_2025//data//processed//PulseDB//PulseDB_Vital//p000{j}.csv"

    if not os.path.exists(file_path):
        continue
    
    try:
        df = pd.read_csv(file_path)

        if any('HDF5 dataset' in col for col in df.columns):
            df.columns = df.columns.map(lambda col: re.search(r'"(.*?)"', col).group(1) if 'HDF5 dataset' in col else col)

        selected_columns = [
            "PPG_F", "ABP_F", "Age", "Gender", "Height", "Weight", "BMI", "T", "SegSBP", "SegDBP"
        ]
        df = df[selected_columns]

        df['Label'] = np.select(
            [
                (df['SegSBP'] < 120) & (df['SegDBP'] < 80),
                (df['SegSBP'] >= 120) & (df['SegSBP'] < 130) & (df['SegDBP'] < 80),
                (df['SegSBP'] >= 130) & (df['SegSBP'] < 140) & (df['SegDBP'] >= 80) & (df['SegDBP'] < 90),
                (df['SegSBP'] >= 140) & (df['SegDBP'] >= 90),
                (df['SegSBP'] > 180) | (df['SegDBP'] > 120)
            ],
            [
                'Normal BP', 
                'Elevated BP', 
                'Hypertension Stage 1', 
                'Hypertension Stage 2', 
                'Hypertensive Crisis'
            ],
            default='Normal BP'  # Default value if none of the conditions match
        )
        df.to_csv(output_path,index=False)
        print(f"Saved: {output_path}")

        os.remove(file_path)

    except Exception as e:
        print(f"Skipping {file_path} due to error: {e}")

In [ ]:
# After the loop ends
processed_dir = "E:/CNR_2025/data/processed/PulseDB/PulseDB_Vital"


csv_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.endswith(".csv")]
print(csv_files[-1])
print(len(csv_files))
for i in range(int(len(csv_files)/23)):
    all_dfs = []
    union_output_path = f"E:/CNR_2025/data/processed/PulseDB/union_PulseDB_Vital-{i+1}.parquet"
    for csv_file in csv_files[i*23:(i+1)*23]:
        try:
            df = pd.read_csv(csv_file)
            all_dfs.append(df)
        except Exception as e:
            print(f"Skipping {csv_file} due to error: {e}")
    # Concatenate and save the final union CSV
    if all_dfs:
        union_df = pd.concat(all_dfs, ignore_index=True)
        union_df.to_parquet(union_output_path, index=False,engine='pyarrow')
        print(f"Union {i+1}/12 saved to: {union_output_path}")


In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.utils import resample

processed_dir = "E:/CNR_2025/data/processed/PulseDB/"

# List all .parquet files
parquet_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.endswith(".parquet")]
print(parquet_files[-1])
i=0
for f in parquet_files:
    i+=1
    union_output_path=f"E:/CNR_2025/data/processed/PulseDB/union_PulseDB_Vital-0{i}.parquet"
    try:
        df = pd.read_parquet(f)
    except Exception as e:
        print(f"Skipping {f} due to error: {e}")
        continue
    
    df_majority = df[df["Label"] == "Normal BP"]
    df_minority = df[df["Label"] != "Normal BP"]
    
    minority_count = len(df_minority)
    
    df_majority_downsampled = resample(
        df_majority,
        replace=False,
        n_samples=minority_count,
        random_state=42
    )
    
    df_balanced = pd.concat([df_majority_downsampled, df_minority]).sample(frac=1, random_state=42)
    df_balanced.to_parquet(union_output_path, index=False,engine='pyarrow')
    print(f"Union {i+1}/12 saved to: {union_output_path}")

